In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from collections import namedtuple, deque
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import ns3ai_multibss_py as py_binding
from ns3ai_utils import Experiment
import sys
import traceback

ModuleNotFoundError: No module named 'ns3ai_multibss_py'

In [2]:
#Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

#Define valid transitions for interaction between agent and environment
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

In [3]:
class ReplayMemory(object):
    '''
        Defines a memory queue storing action-reward pairs for the model
    '''
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
class simple_DQN(nn.Module):
    '''
        Simple Multi-layer perceptron model with one hidden layer to generate Q-function
    '''
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, n_actions)

    #Called with either one element to determine next action, or a batch during optimization. 
    #Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)


In [11]:
class DQN(nn.module):
    '''
        Bigger MLP with four hidden layers
    '''
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 256)
        self.layer2 = nn.Linear(256, 512)
        self.layer3 = nn.Linear(512, 1024)
        self.layer4 = nn.Linear(1024, 512)
        self.layer5 = nn.Linear(512, 256)
        self.layer6 = nn.Linear(256, n_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = F.relu(self.layer5(x))
        return self.layer6(x)

AttributeError: module 'torch.nn' has no attribute 'module'

In [7]:
#Define simulation parameters to call ns3 sim
ns3Settings = {
    'pktSize': 1500,
    'duration': 100,
    'gi': 800,
    'channelWidth': 20,
    'rng': 2,
    'apNodes': 4,
    'networkSize': 4,
    'ring': 0,
    'maxMpdus': 5,
    'autoMCS': True,
    'prop': 'tgax',
    'app': 'setup-done',
    'pktInterval': 5000,
    'boxsize': 25,
    'drl': True,
    'configFile': 'contrib/ai/examples/multi-bss/config.txt',
}

n_ap = int(ns3Settings['apNodes'])
n_sta = int(ns3Settings['networkSize'])
n_total = n_ap * (n_sta + 1)
state = np.zeros((n_sta + 1, n_total + 1))
rewards = []
overall_rewards = []

BATCH_SIZE = 32
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 50
TAU = 0.005
LR = 1e-4

In [9]:
#Define the action space range for the agent
CCA_SENSITIVITY_RANGE = [-82, -62]
CHANNEL_WIDTH_OPTIONS = [20, 40, 80, 160]
CHANNEL_NUMBER_OPTIONS = [36, 42, 48, 149]
N_BSS = 4

#Calculates the dimensions of the action space
def get_action_space(CCA_range, channel_width_range, channel_no_range, n_BSS):
    return len(range(*CCA_range)) * len(channel_width_range) * len(channel_no_range) * n_BSS

#Compute CCA range, Channel width, and Channel number given action
def decode_action(action):
    cca_range = len(range(*CCA_SENSITIVITY_RANGE))
    width_range = len(CHANNEL_WIDTH_OPTIONS)

    cca_index = action % cca_range
    width_index = (action // cca_range) % width_range
    channel_index = action // (cca_range * width_range)

    return CCA_SENSITIVITY_RANGE[0] + cca_index, CHANNEL_WIDTH_OPTIONS[width_index], CHANNEL_NUMBER_OPTIONS[channel_index]



In [10]:
#Generates a configuration file to run the experiment given parameters
def generate_config_file(bss_params):
    config_template = """#NodeId:TrafficType,CcaSensitivity,TxPower,ChannelWidth,ChannelNumber

    #BSS-0
    #AP
    0:none,{cca_0},{tx_power_0},{width_0},{channel_0}
    #STA
    4:constant,{cca_0},{tx_power_0},{width_0},{channel_0}
    8:constant,{cca_0},{tx_power_0},{width_0},{channel_0}
    12:constant,{cca_0},{tx_power_0},{width_0},{channel_0}
    16:constant,{cca_0},{tx_power_0},{width_0},{channel_0}

    #BSS-1
    #AP
    1:none,{cca_1},{tx_power_1},{width_1},{channel_1}
    #STA
    5:constant,{cca_1},{tx_power_1},{width_1},{channel_1}
    9:constant,{cca_1},{tx_power_1},{width_1},{channel_1}
    13:constant,{cca_1},{tx_power_1},{width_1},{channel_1}
    17:constant,{cca_1},{tx_power_1},{width_1},{channel_1}

    #BSS-2
    #AP
    2:none,{cca_2},{tx_power_2},{width_2},{channel_2}
    #STA
    6:constant,{cca_2},{tx_power_2},{width_2},{channel_2}
    10:constant,{cca_2},{tx_power_2},{width_2},{channel_2}
    14:constant,{cca_2},{tx_power_2},{width_2},{channel_2}
    18:constant,{cca_2},{tx_power_2},{width_2},{channel_2}

    #BSS-3
    #AP
    3:none,{cca_3},{tx_power_3},{width_3},{channel_3}
    #STA
    7:constant,{cca_3},{tx_power_3},{width_3},{channel_3}
    11:constant,{cca_3},{tx_power_3},{width_3},{channel_3}
    15:constant,{cca_3},{tx_power_3},{width_3},{channel_3}
    19:constant,{cca_3},{tx_power_3},{width_3},{channel_3}
    """
    for i in range(bss_params):
        formatted_params = {
            f"cca_{i}": bss_params[i]['cca'],
            f"tx_power_{i}": bss_params[i]['tx_power'],
            f"width_{i}": bss_params[i]['width'],
            f"channel_{i}": bss_params[i]['channel']
        }

    with open(ns3Settings['configFile'], 'w') as config_file:
        config_file.write(config_template.format(**formatted_params))
        

In [ ]:
#Define action and observation states, build model
n_actions = get_action_space(CCA_SENSITIVITY_RANGE, CHANNEL_WIDTH_OPTIONS, CHANNEL_NUMBER_OPTIONS, N_BSS)
n_observations = (n_sta + 1) * (n_total + 1)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(200)

steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[np.random.randint(0, n_actions)]], device=device, dtype=torch.long)
    


In [ ]:
#Now we train the model
#Unfairness penalization factor
alpha = 10E-3
try:
    exp = Experiment("ns3ai_multibss", "../../../../", py_binding, handleFinish=True, useVector=True, vectorSize=n_total)
    msgInterface = exp.run(setting=ns3Settings, show_output=True)

    while True:
        msgInterface.PyRecvBegin()
        if msgInterface.PyGetFinished():
            print("Finished")
            break
        #Calculate system aggregate throughput
        throughput = []
        for i in range(n_total):
            txNode = msgInterface.GetCpp2PyVector()[i].txNode
            for j in range(n_sta + 1):
                state[j, txNode] = msgInterface.GetCpp2PyVector()[i].rxPower[j]
            throughput.append(msgInterface.GetCpp2PyVector()[i].throughput)
        msgInterface.PyRecvEnd()

        cur_state = torch.tensor(state.reshape(1, -1)[0], dtype=torch.float32, device=device).unsqueeze(0)
        if steps_done == 0:
            prev_state = cur_state
            action = torch.tensor([[0]], device=device, dtype=torch.long)
        else:
            #IMPLEMENTS REWARD FUNCTION
            tpt = np.asarray(throughput)
            mean_tpt = np.mean(tpt)
            reward = mean_tpt - (alpha/np.size(tpt)) * np.sum(tpt - mean_tpt)
            
            rewards.append(reward)
            reward = torch.tensor([reward], device=device)
            memory.push(prev_state, action, cur_state, reward)
            prev_state = cur_state
            action = select_action(cur_state)

        cca_sensitivity, channel_width, channel_number = decode_action(action.item())
        bss_params = [
            {"cca": cca_sensitivity, "tx_power": 12, "width": channel_width, "channel": channel_number}
            for _ in range(n_ap)
        ]
        generate_config_file(bss_params)

        msgInterface.PySendBegin()
        msgInterface.GetPy2CppVector()[0].newCcaSensitivity = cca_sensitivity
        msgInterface.GetPy2CppVector()[0].newChannelWidth = channel_width
        msgInterface.GetPy2CppVector()[0].newChannelNumber = channel_number
        msgInterface.PySendEnd()

except Exception as e:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print("Exception occurred: {}".format(e))
    traceback.print_tb(exc_traceback)
    exit(1)

finally:
    print("Exiting...")

